In [15]:
import json
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier


file = '../data_resources/topics/vng_training.json'
data = json.load(open(file))

In [16]:
transformer = TfidfVectorizer(smooth_idf=False)

corpus = []
y = []

for obj in data:
    content = obj['content']
    corpus.append(content)
    
    y.append(obj['sub_topic'])
    

X = transformer.fit_transform(corpus)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Classifiers experimented with and score achieved:
classifiers = {
    'knn': KNeighborsClassifier(3), # 0.82
    'svm1': SVC(kernel="linear", C=0.25, probability=True), # 0.82
    'svm2': SVC(gamma=2, C=1), # Took a long time - 0.13
    'gpc': GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True), # 0.
    'tree': DecisionTreeClassifier(max_depth=75), # 0.80
    'randomf': RandomForestClassifier(max_depth=30, n_estimators=100, max_features=1000), # 0.82
    'mlp': MLPClassifier(alpha=1), # 0.85
    'ada': AdaBoostClassifier(), # 0.43
    'ldc': LinearDiscriminantAnalysis(solver = 'svd', n_components = 1),
    'gnb': GaussianNB(), # 0.70
    'qdc': QuadraticDiscriminantAnalysis() # 0.80
}

In [29]:
%time clf = OneVsRestClassifier(classifiers['randomf']).fit(X_train, y_train)

y_pred = clf.predict(X_test)

score = accuracy_score(y_test, y_pred)
print(score)

CPU times: user 3min 5s, sys: 31.8 ms, total: 3min 5s
Wall time: 3min 5s
0.6125
